In [1]:
(require booleans redex/reduction-semantics)

In [2]:
(define/ppl start #:no-check
  ((r0 = GO)
   ;; pause
   (l1 = GO)
   (l0 = (and RES reg-out))
   (SEL = reg-out)
   (reg-in = (and (not KILL) reg-set))
   (reg-set = (or is-susp GO))
   (is-susp = (and SUSP reg-out))
    ;; synchronizer
    (K1 = (and left1 (and both1 right1)))
    (K0 = (and left0 (and both0 right0)))
    (left1 = (or left0 l1))
    (right1 = (or right0 false))
    (both1 = (or l1 false))
    (left0 = (or l0 lem))
    (right0 = (or r0 (not GO)))
    (both0 = (or l0 r0))
    (lem = (not  (or reg-out GO)))))
    

(r0 = GO)
(l1 = GO)
(l0 = (and RES reg-out))
(SEL = reg-out)
(reg-in = (and (not KILL) reg-set))
(reg-set = (or is-susp GO))
(is-susp = (and SUSP reg-out))
(K1 = (and left1 (and both1 right1)))
(K0 = (and left0 (and both0 right0)))
(left1 = (or left0 l1))
(right1 = (or right0 false))
(both1 = (or l1 false))
(left0 = (or l0 lem))
(right0 = (or r0 (not GO)))
(both0 = (or l0 r0))
(lem = (not (or reg-out GO)))


In [3]:
(define/ppl end #:no-check
   ((K1 = GO)
   (K0 = (and RES reg-out))
   (SEL = reg-out)
   (reg-in = (and (not KILL) reg-set))
   (reg-set = (or is-susp GO))
   (is-susp = (and SUSP reg-out))))

(K1 = GO)
(K0 = (and RES reg-out))
(SEL = reg-out)
(reg-in = (and (not KILL) reg-set))
(reg-set = (or is-susp GO))
(is-susp = (and SUSP reg-out))


In [14]:
(assert-same (term (replace* start [GO ⊥] [reg-out false]))
             (term (replace* end [GO ⊥] [reg-out false])))

equal-class: the diff:
 '((("a broken clause:" (K0 = false))
   ((RES = true) (KILL = true) (SUSP = true))
   ((r0 = ⊥)
    (l1 = ⊥)
    (l0 = false)
    (SEL = false)
    (reg-in = false)
    (reg-set = ⊥)
    (is-susp = false)
    (K1 = ⊥)
    (K0 = ⊥)
    (left1 = ⊥)
    (right1 = ⊥)
    (both1 = ⊥)
    (left0 = ⊥)
    (right0 = ⊥)
    (both0 = ⊥)
    (lem = ⊥))
   ((K1 = ⊥)
    (K0 = false)
    (SEL = false)
    (reg-in = false)
    (reg-set = ⊥)
    (is-susp = false)))
  (("a broken clause:" (K0 = false))
   ((RES = true) (KILL = true) (SUSP = false))
   ((r0 = ⊥)
    (l1 = ⊥)
    (l0 = false)
    (SEL = false)
    (reg-in = false)
    (reg-set = ⊥)
    (is-susp = false)
    (K1 = ⊥)
    (K0 = ⊥)
    (left1 = ⊥)
    (right1 = ⊥)
    (both1 = ⊥)
    (left0 = ⊥)
    (right0 = ⊥)
    (both0 = ⊥)
    (lem = ⊥))
   ((K1 = ⊥)
    (K0 = false)
    (SEL = false)
    (reg-in = false)
    (reg-set = ⊥)
    (is-susp = false)))
  (("a broken clause:" (K0 = false))
   ((RES = true) (KILL = tru

    (is-susp = false))))
  context...:
   eval-one-top12
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:510:0: call-with-custodian-shutdown
   /Applications/Racket/racket/collects/racket/private/more-scheme.rkt:148:2: call-with-break-parameterization
   .../more-scheme.rkt:261:28
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:878:5: loop


In [5]:
(define/ppl step1 #:no-check
  (convert-P (replace* start [GO ⊥] [reg-out false])))

((+ r0) = false)
((- r0) = false)
((+ l1) = false)
((- l1) = false)
((+ l0) = (and (+ RES) false))
((- l0) = (or (- RES) true))
((+ SEL) = false)
((- SEL) = true)
((+ reg-in) = (and (- KILL) (+ reg-set)))
((- reg-in) = (or (+ KILL) (- reg-set)))
((+ reg-set) = (or (+ is-susp) false))
((- reg-set) = (and (- is-susp) false))
((+ is-susp) = (and (+ SUSP) false))
((- is-susp) = (or (- SUSP) true))
((+ K1) = (and (+ left1) (and (+ both1) (+ right1))))
((- K1) = (or (- left1) (or (- both1) (- right1))))
((+ K0) = (and (+ left0) (and (+ both0) (+ right0))))
((- K0) = (or (- left0) (or (- both0) (- right0))))
((+ left1) = (or (+ left0) (+ l1)))
((- left1) = (and (- left0) (- l1)))
((+ right1) = (or (+ right0) false))
((- right1) = (and (- right0) true))
((+ both1) = (or (+ l1) false))
((- both1) = (and (- l1) true))
((+ left0) = (or (+ l0) (+ lem)))
((- left0) = (and (- l0) (- lem)))
((+ right0) = (or (+ r0) false))
((- right0) = (and (- r0) false))
((+ both0) = (or (+ l0) (+ r0)))
((- both0) 

In [6]:
(define/ppl step2 
  propigate/remove* step1 (+ r0) (- r0) (- l1) (+ l1) (+ SEL) (- SEL) (+ l0) (- l0))

((+ reg-in) = (and (- KILL) (+ reg-set)))
((- reg-in) = (or (+ KILL) (- reg-set)))
((+ reg-set) = (or (+ is-susp) false))
((- reg-set) = (and (- is-susp) false))
((+ is-susp) = (and (+ SUSP) false))
((- is-susp) = (or (- SUSP) true))
((+ K1) = (and (+ left1) (and (+ both1) (+ right1))))
((- K1) = (or (- left1) (or (- both1) (- right1))))
((+ K0) = (and (+ left0) (and (+ both0) (+ right0))))
((- K0) = (or (- left0) (or (- both0) (- right0))))
((+ left1) = (or (+ left0) false))
((- left1) = (and (- left0) false))
((+ right1) = (or (+ right0) false))
((- right1) = (and (- right0) true))
((+ both1) = (or false false))
((- both1) = (and false true))
((+ left0) = (or (and (+ RES) false) (+ lem)))
((- left0) = (and (or (- RES) true) (- lem)))
((+ right0) = (or false false))
((- right0) = (and false false))
((+ both0) = (or (and (+ RES) false) false))
((- both0) = (and (or (- RES) true) false))
((+ lem) = (and true false))
((- lem) = (or false false))


In [7]:
(define/ppl step3 
  replace* step2
  [(and (+ RES) false) false]
  [(or (- RES) true) true]
  [(or (+ is-susp) false) (+ is-susp)]
  [(and (- is-susp) false) false]
  [(and (+ SUSP) false) false]
  [(or (- SUSP) true) true]
  [(or (+ left0) false) (+ left0)]
  [(and (- left0) false) false]
  [(or (+ right0) false) (+ right0)]
  [(and (- right0) true) (- right0)]
  [(or false false) false]
  [(and false true) false]
  [(and false false) false]
  [(or (+ l0) false) (+ l0)]
  [(and (- l0) false) false]
  [(and true false) false])

((+ reg-in) = (and (- KILL) (+ reg-set)))
((- reg-in) = (or (+ KILL) (- reg-set)))
((+ reg-set) = (+ is-susp))
((- reg-set) = false)
((+ is-susp) = false)
((- is-susp) = true)
((+ K1) = (and (+ left1) (and (+ both1) (+ right1))))
((- K1) = (or (- left1) (or (- both1) (- right1))))
((+ K0) = (and (+ left0) (and (+ both0) (+ right0))))
((- K0) = (or (- left0) (or (- both0) (- right0))))
((+ left1) = (+ left0))
((- left1) = false)
((+ right1) = (+ right0))
((- right1) = (- right0))
((+ both1) = false)
((- both1) = false)
((+ left0) = (or false (+ lem)))
((- left0) = (and true (- lem)))
((+ right0) = false)
((- right0) = false)
((+ both0) = false)
((- both0) = false)
((+ lem) = false)
((- lem) = false)


In [8]:
(define/ppl step4
  propigate/remove* step3
  (+ reg-set) (- reg-set)
  (+ is-susp) (- is-susp) (+ right1) (- right1) (+ both1) (- both1) (+ left0) (- left0)
  (+ right0) (- right0) (+ both0) (- both0) (+ lem) (- lem))

((+ reg-in) = (and (- KILL) false))
((- reg-in) = (or (+ KILL) false))
((+ K1) = (and (+ left1) (and false false)))
((- K1) = (or (- left1) (or false false)))
((+ K0) = (and (or false false) (and false false)))
((- K0) = (or (and true false) (or false false)))
((+ left1) = (or false false))
((- left1) = false)


In [9]:
(define/ppl step5
  replace* step4
  [(and (- KILL) false) false]
  [(or (+ KILL) false) (+ KILL)]
  [(and (+ left1) (and false false)) false]
  [(or (- left1) (or false false)) (- left1)]
  [(and (or false false) (and false false)) false]
  [(or (and true false) (or false false)) false]
  [(or false false) false])

((+ reg-in) = false)
((- reg-in) = (+ KILL))
((+ K1) = false)
((- K1) = (- left1))
((+ K0) = false)
((- K0) = false)
((+ left1) = false)
((- left1) = false)


In [10]:
(define/ppl step6
  propigate/remove* step5
  (+ reg-in) (- reg-in) (+ left1) (- left1))

((+ K1) = false)
((- K1) = false)
((+ K0) = false)
((- K0) = false)


In [11]:
(define/ppl step-a #:no-check
  (replace* end [GO ⊥] [reg-out false]))

(K1 = ⊥)
(K0 = (and RES false))
(SEL = false)
(reg-in = (and (not KILL) reg-set))
(reg-set = (or is-susp ⊥))
(is-susp = (and SUSP false))


In [12]:
(define/ppl step-b
  replace* step-a
  [(and RES false) false]
  [(and SUSP false) false])

(K1 = ⊥)
(K0 = false)
(SEL = false)
(reg-in = (and (not KILL) reg-set))
(reg-set = (or is-susp ⊥))
(is-susp = false)
